Artifical Start

In [121]:
using LinearAlgebra
using SparseArrays
using LightGraphs
using SimpleWeightedGraphs
using GraphPlot
using BenchmarkTools
using Random
include("minCost.jl")

AddToTable!

In [ ]:
# Lexicographical

In [136]:
nodes = 5
E = [(1,2) (1,3) (1,5) (2,3) (3,4) (4,2) (4,5) (5,3)]
cost_dict = Dict((1,2)=> 1,(1,3)=> 2, (1,5)=> 1, (2,3)=> 2, (3,4)=> 1, (4,2)=> 1, (4,5)=> 1, (5,3)=> 1)
## Artifical Start
b = Int64[2;5;1;-4;-4;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost

#############################################################################################

nodes = 5
E = [(1,2) (1,3) (1,4) (2,5) (3,5) (4,5)]
cost_dict = Dict((1,2)=>1, (1,3)=>1, (1,4)=>1, (2,5)=>2, (3,5)=>2, (4,5)=>1)
## Artifical Start
b = Int64[5;-1;-1;-1;-2;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost

6-element Array{Int64,1}:
 1
 1
 1
 1
 1
 0

In [137]:
n = nodes + 1 
x = abs.(b)
B = ArtificalBase_table(b)
B_edgeList = ArtificalBase_list(b)
B_matrix = BaseMatrix(B_edgeList,n)
NB_edgeList = copy(E)

1×6 Array{Tuple{Int64,Int64},2}:
 (1, 2)  (1, 3)  (1, 4)  (2, 5)  (3, 5)  (4, 5)

In [139]:
Matrix(B_matrix)

6×6 Array{Int64,2}:
  1   0   0   0   0   0
  0  -1   0   0   0   0
  0   0  -1   0   0   0
  0   0   0  -1   0   0
  0   0   0   0  -1   0
 -1   1   1   1   1  -1

In [124]:
w = zeros(n,1)
valid = true
while (sum(c) != 0)
    w = c'/B_matrix    
    println("Dual: $w")
    max ,enteringEdge = ReducedCost(w,NB_edgeList,1, 0)
    
    println("Entering:  $max , $enteringEdge")
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,n,B)
    println("Entering and Path ($edge_u ,$edge_v)", path)
    Flow_list, valid = Get_flow(path)
    println(Flow_list, valid)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        println("Exiting: $exitingEdge")
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,n,B)
        RemoveFromTable!(edge_j,edge_i,n,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        Update_BaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        c[min_index] = 0
    end
    for (i,e) in enumerate(NB_edgeList) 
        if e == enteringEdge
            NB_edgeList[i] = (0,0)
            break
        end
    end
end

Dual: [1.0 -1.0 -1.0 -1.0 -1.0 -0.0]
(1, 2): 2.0 =  2.0
(1, 3): 2.0 =  2.0
(1, 4): 2.0 =  2.0
(2, 5): 0.0 =  0.0
(3, 5): 0.0 =  0.0
(4, 5): 0.0 =  0.0
Entering:  2.0 , (1, 2)
Entering and Path (1 ,2)[(1, 2), (1, 6), (6, 2)]
[1, -1, -1]true
Exiting: (6, 2)
Dual: [1.0 1.0 -1.0 -1.0 -1.0 -0.0]
(1, 3): 2.0 =  2.0
(1, 4): 2.0 =  2.0
(2, 5): 2.0 =  2.0
(3, 5): 0.0 =  0.0
(4, 5): 0.0 =  0.0
Entering:  2.0 , (1, 3)
Entering and Path (1 ,3)[(1, 3), (1, 6), (6, 3)]
[1, -1, -1]true
Exiting: (6, 3)
Dual: [1.0 1.0 1.0 -1.0 -1.0 -0.0]
(1, 4): 2.0 =  2.0
(2, 5): 2.0 =  2.0
(3, 5): 2.0 =  2.0
(4, 5): 0.0 =  0.0
Entering:  2.0 , (1, 4)
Entering and Path (1 ,4)[(1, 4), (1, 6), (6, 4)]
[1, -1, -1]true
Exiting: (6, 4)
Dual: [1.0 1.0 1.0 1.0 -1.0 -0.0]
(2, 5): 2.0 =  2.0
(3, 5): 2.0 =  2.0
(4, 5): 2.0 =  2.0
Entering:  2.0 , (2, 5)
Entering and Path (2 ,5)[(2, 5), (1, 2), (1, 6), (6, 5)]
[1, 1, -1, -1]true
Exiting: (6, 5)
Dual: [1.0 1.0 1.0 1.0 1.0 -0.0]
(3, 5): 0.0 =  0.0
(4, 5): 0.0 =  0.0
Entering:  0.0

In [125]:
basicEdge =  Base_list(B_edgeList, nodes)
cost = Set_cost(basicEdge, cost_dict)
NB_edgeList = NonBasic_list(basicEdge, E)
B_matrix = BaseMatrix(basicEdge,nodes)
B = Base_table(basicEdge, nodes)

b = b[1:end-1] # Flow
x = x[1:end-1]

(3, 5)
(4, 5)


5-element Array{Int64,1}:
 0
 3
 1
 1
 2

In [126]:
NB_edgeList

2×1 Array{Tuple{Int64,Int64},2}:
 (3, 5)
 (4, 5)

In [127]:
basicEdge

5×1 Array{Tuple{Int64,Int64},2}:
 (1, 1)
 (1, 2)
 (1, 3)
 (1, 4)
 (2, 5)

In [128]:
cost'x

1-element Array{Float64,1}:
 9.0

In [129]:
B_matrix*x == b

true

In [130]:
for i in 1:nodes
    println("$(basicEdge[i]) : $(x[i])  $(cost[i])")
end

(1, 1) : 0  0.0
(1, 2) : 3  1.0
(1, 3) : 1  1.0
(1, 4) : 1  1.0
(2, 5) : 2  2.0


In [131]:
w = cost'/B_matrix
while (w*cost != 0)
    
    max ,enteringEdge = ReducedCost(w,NB_edgeList, 2, cost_dict)
    println("Entering:  $max , $enteringEdge")
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,nodes,B)
    println("Entering and Path ($edge_u ,$edge_v)", path)
    Flow_list, valid = Get_flow(path)
    if valid
        indexes = Get_Indexes(path,basicEdge) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        println("Exiting: $exitingEdge")
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,nodes,B)
        RemoveFromTable!(edge_j,edge_i,nodes,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        Update_BaseMatrix!(B_matrix,basicEdge,exitingEdge,enteringEdge)
        cost[min_index] = cost_dict[enteringEdge]
    end
    w = cost'/B_matrix
end

(3, 5): 0.0 =  0.0
(4, 5): 1.0 =  1.0
Entering:  1.0 , (4, 5)
Entering and Path (4 ,5)[(4, 5), (1, 4), (1, 2), (2, 5)]
Exiting: (2, 5)
(3, 5): -1.0 =  -1.0
(4, 5): 0.0 =  0.0
Entering:  0.0 , (4, 5)
ENDING!!


In [132]:
E

1×6 Array{Tuple{Int64,Int64},2}:
 (1, 2)  (1, 3)  (1, 4)  (2, 5)  (3, 5)  (4, 5)

In [133]:
basicEdge

5×1 Array{Tuple{Int64,Int64},2}:
 (1, 1)
 (1, 2)
 (1, 3)
 (1, 4)
 (4, 5)

In [134]:
x

5-element Array{Int64,1}:
 0
 1
 1
 3
 2

In [135]:
NB_edgeList = NonBasic_list(basicEdge, E)
ReducedCost(w,NB_edgeList, 2, cost_dict)

(2, 5)
(3, 5)
(2, 5): -1.0 =  -1.0
(3, 5): -1.0 =  -1.0


(-1.0, (2, 5))

In [103]:
w*B_matrix - cost'

1×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0

In [91]:
w*b 

1-element Array{Float64,1}:
 7.0

In [97]:
w

1×5 Array{Float64,2}:
 -0.0  -1.0  -1.0  -1.0  -2.0

In [98]:
x

5-element Array{Int64,1}:
 0
 1
 1
 3
 2

In [92]:
cost'x

1-element Array{Float64,1}:
 7.0

In [93]:
x

5-element Array{Int64,1}:
 0
 1
 1
 3
 2

In [94]:
w

1×5 Array{Float64,2}:
 -0.0  -1.0  -1.0  -1.0  -2.0

In [95]:
B_matrix*x 

5-element Array{Int64,1}:
  5
 -1
 -1
 -1
 -2

In [96]:
 B_matrix

5×5 SparseMatrixCSC{Int64,Int64} with 10 stored entries:
  [1, 1]  =  -1
  [1, 2]  =  1
  [2, 2]  =  -1
  [1, 3]  =  1
  [3, 3]  =  -1
  [1, 4]  =  1
  [4, 4]  =  -1
  [2, 5]  =  0
  [4, 5]  =  1
  [5, 5]  =  -1

In [386]:
#Check Feasibility
(w*B_matrix)' .>= cost

5×1 BitMatrix:
 1
 1
 1
 1
 1

In [387]:
w 

1×5 Matrix{Float64}:
 0.0  -0.0  -2.0  -3.0  -4.0

In [388]:
for i in 1:nodes
    println("$(basicEdge[i]) : $(x[i])  $(cost[i])")
end

(1, 3) : 2  2.0
(2, 2) : 0  0.0
(3, 4) : 8  1.0
(2, 3) : 5  2.0
(4, 5) : 4  1.0


In [333]:
#Min Cost
cost'x

1-element Vector{Float64}:
 7.0

In [334]:
w*B_matrix

1×5 Matrix{Float64}:
 0.0  1.0  1.0  1.0  1.0

In [335]:
cost

5×1 Matrix{Float64}:
 0.0
 1.0
 1.0
 1.0
 1.0